3.6 softmax回归的从零开始实现
这一节我们来动手实现softmax回归。首先导入本节实现所需的包或模块。

In [208]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


3.6.1 获取和读取数据
我们将使用Fashion-MNIST数据集，并设置批量大小为256。

In [209]:
from tensorflow.keras.datasets import fashion_mnist
batch_size=256
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = tf.cast(x_train, tf.float32)#在进行矩阵相乘时需要float型，故强制类型转换为float型
x_test=tf.cast(x_test,tf.float32)#在进行矩阵相乘时需要float型，故强制类型转换为float型

In [210]:
train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_iter = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

3.6.2 初始化模型参数
跟线性回归中的例子一样，我们将使用向量表示每个样本。已知每个样本输入是高和宽均为28像素的图像。模型的输入向量的长度是 28×28=784：该向量的每个元素对应图像中每个像素。由于图像有10个类别，单层神经网络输出层的输出个数为10，因此softmax回归的权重和偏差参数分别为784×10和1×10的矩阵。

In [211]:
num_inputs = 784
num_outputs = 10
W = tf.Variable(tf.random.normal([num_inputs, num_outputs],0,0.01,dtype=tf.float32))
b = tf.Variable(tf.zeros(num_outputs),dtype=tf.float32)

3.6.3. 实现softmax运算¶
在介绍如何定义softmax回归之前，我们先描述一下对如何对多维Tensor按维度操作。在下面的例子中，给定一个Tensor矩阵X。我们可以只对其中同一列（axis=0）或同一行（axis=1）的元素求和，并在结果中保留行和列这两个维度（keepdims=True）。

In [212]:
X = tf.constant([[1, 2, 3], [4, 5, 6]])
tf.reduce_sum(X,0,keepdims=True),tf.reduce_sum(X,1,keepdims=True)

(<tf.Tensor: id=17868, shape=(1, 3), dtype=int32, numpy=array([[5, 7, 9]])>,
 <tf.Tensor: id=17870, shape=(2, 1), dtype=int32, numpy=
 array([[ 6],
        [15]])>)

下面我们就可以定义前面小节里介绍的softmax运算了。在下面的函数中，矩阵X的行数是样本数，列数是输出个数。为了表达样本预测各个输出的概率，softmax运算会先通过exp函数对每个元素做指数运算，再对exp矩阵同行元素求和，最后令矩阵每行各元素与该行元素之和相除。这样一来，最终得到的矩阵每行元素和为1且非负。因此，该矩阵每行都是合法的概率分布。softmax运算的输出矩阵中的任意一行元素代表了一个样本在各个输出类别上的预测概率

In [213]:
def softmax(X):
    X_exp = tf.exp(X)
    partition = tf.reduce_sum(X_exp,axis=1, keepdims=True)
    return X_exp / partition  # 这里应用了广播机制

可以看到，对于随机输入，我们将每个元素变成了非负数，且每一行和为1。

In [214]:
X = tf.random.normal(shape=(2, 5))
X_prob = softmax(X)
index=tf.argmax(X_prob,axis=1)
X_prob, tf.reduce_sum(X_prob,axis=1),index

(<tf.Tensor: id=17880, shape=(2, 5), dtype=float32, numpy=
 array([[0.08454549, 0.35096657, 0.29147166, 0.18779236, 0.08522389],
        [0.1605916 , 0.2528661 , 0.09678561, 0.12512548, 0.36463124]],
       dtype=float32)>,
 <tf.Tensor: id=17884, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>,
 <tf.Tensor: id=17882, shape=(2,), dtype=int64, numpy=array([1, 4], dtype=int64)>)

3.6.4. 定义模型¶
有了softmax运算，我们可以定义上节描述的softmax回归模型了。这里通过reshape函数将每张原始图像改成长度为num_inputs的向量。

In [215]:
def net(X):
    return softmax(tf.cast(tf.matmul(tf.reshape(X,(-1, num_inputs)), W) + b,dtype=tf.double))

3.6.5. 定义损失函数¶
上一节中，我们介绍了softmax回归使用的交叉熵损失函数。为了得到标签的预测概率，我们可以使用pick函数。在下面的例子中，变量y_hat是2个样本在3个类别的预测概率，变量y是这2个样本的标签类别。通过使用pick函数，我们得到了2个样本的标签的预测概率。与“softmax回归”一节数学表述中标签类别离散值从1开始逐一递增不同，在代码中，标签类别的离散值是从0开始逐一递增的。

下面实现了“softmax回归”一节中介绍的交叉熵损失函数。

In [222]:
y_hat = np.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = np.array([0, 2], dtype='int32')

In [223]:
def cross_entropy(y_hat, y):
        y_hat=tf.argmax(y_hat,axis=1)
        print(y_hat.shape,y.shape)
        y=tf.cast(y,dtype=tf.float32)
        return tf.losses.categorical_crossentropy(y_hat,y)

3.6.6 计算分类准确率
给定一个类别的预测概率分布y_hat，我们把预测概率最大的类别作为输出类别。如果它与真实类别y一致，说明这次预测是正确的。分类准确率即正确预测数量与总预测数量之比。

为了演示准确率的计算，下面定义准确率accuracy函数。其中tf.argmax(dim=1)返回矩阵y_hat每行中最大元素的索引，且返回结果与变量y形状相同。相等条件判断式(y_hat.argmax(dim=1) == y)是一个类型为ByteTensor的Tensor，我们用float()将其转换为值为0（相等为假）或1（相等为真）的浮点型Tensor。

In [224]:
def accuracy(y_hat, y):
    return np.mean((tf.argmax(y_hat,axis=1) == y))

让我们继续使用在演示pick函数时定义的变量y_hat和y，并将它们分别作为预测概率分布和标签。可以看到，第一个样本预测类别为2（该行最大元素0.6在本行的索引为2），与真实标签0不一致；第二个样本预测类别为2（该行最大元素0.5在本行的索引为2），与真实标签2一致。因此，这两个样本上的分类准确率为0.5。

In [225]:
accuracy(y_hat, y)

0.5

类似地，我们可以评价模型net在数据集data_iter上的准确率

In [228]:
# 描述,对于tensorflow2中，比较的双方必须类型都是int型，所以要将输出和标签都转为int型
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for _, (X, y) in enumerate(data_iter):
        y=tf.cast(y,dtype=tf.int64)
        acc_sum += np.sum(tf.cast(tf.argmax(net(X),axis=1),dtype=tf.int64) == y)
#         print(acc_sum.shape,y.shape,acc_sum.dtype,y.dtype)
        n += y.shape[0]
    return acc_sum / n

In [229]:
print(evaluate_accuracy(test_iter, net))

0.1072


3.6.7. 训练模型¶
训练softmax回归的实现跟“线性回归的从零开始实现”一节介绍的线性回归中的实现非常相似。我们同样使用小批量随机梯度下降来优化模型的损失函数。在训练模型时，迭代周期数num_epochs和学习率lr都是可以调的超参数。改变它们的值可能会得到分类更准确的模型。

In [230]:
for X, y in test_iter:
    break

true_labels = get_fashion_mnist_labels(y.numpy())
pred_labels = get_fashion_mnist_labels((tf.argmax(net(X),axis=1)).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

show_fashion_mnist(X[0:9], titles[0:9])

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'reshape'